In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [123]:
df_x=  pd.read_csv('C:/Users/bhara/Downloads/x_train.csv')
df_y =  pd.read_csv('C:/Users/bhara/Downloads/y_train.csv')
df_x_test =  pd.read_csv('C:/Users/bhara/Downloads/x_test.csv')

In [124]:
x_input_text = df_x['text'].values
x_input_text

array(['Oh and I forgot to also mention the weird color effect it has on your phone.',
       "THAT one didn't work either.", 'Waste of 13 bucks.', ...,
       'Ambience is perfect.',
       'We ordered the duck rare and it was pink and tender on the inside with a nice char on the outside.',
       'Service was good and the company was better!'], dtype=object)

In [125]:
XTest = df_x_test['text'].values
XTest

array(['It only recognizes the Phone as its storage device.',
       'Disappointing accessory from a good manufacturer.',
       "The one big drawback of the MP3 player is that the buttons on the phone's front cover that let you pause and skip songs lock out after a few seconds.",
       'This particular model would not work with my Motorola Q smartphone.',
       'If the two were seperated by a mere 5+ ft I started to notice excessive static and garbled sound from the headset.',
       "Also the area where my unit broke).- I'm not too fond of the magnetic strap.",
       'What possesed me to get this junk, I have no idea...',
       'The mic there is a joke, and the volume is quite low.',
       'I purchased this and within 2 days it was no longer working!!!!!!!!!',
       'The picture resolution is far below what other comparably-priced phones are offering today.',
       'AFTER ARGUING WITH VERIZON REGARDING THE DROPPED CALLS WE RETURNED THE PHONES AFTER TWO DAYS.',
       'My exper

In [126]:
Y = df_y['is_positive_sentiment'].values
Y

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [127]:
from collections import OrderedDict


word_embeddings = pd.read_csv('C:/Users/bhara/Downloads/glove.6B.50d.txt.zip',
                               header=None, sep=' ', index_col=0,
                               nrows=100000, compression='zip', encoding='utf-8', quoting=3)
word_list = word_embeddings.index.values.tolist()
glove = OrderedDict(zip(word_list, word_embeddings.values))


In [128]:
def get_sentence_embedding(sentence):
    words = sentence.split()
    embeddings = []
    for word in words:
        if word in glove:
            embeddings.append(glove[word])
    if len(embeddings) == 0:
        return np.zeros(50)
    else:
        return np.mean(embeddings, axis=0)

In [129]:
X_train = np.array([get_sentence_embedding(sentence) for sentence in x_input_text])
X_train


array([[ 0.21076848,  0.20281615, -0.15099146, ..., -0.03316823,
        -0.00793963,  0.02360254],
       [ 0.414165  ,  0.306785  , -0.19232   , ..., -0.079209  ,
        -0.27944   ,  0.12079   ],
       [ 0.173215  ,  0.517625  ,  0.1253    , ..., -0.40293   ,
         0.075506  , -0.4325455 ],
       ...,
       [ 0.6185    ,  0.64254   , -0.46552   , ..., -0.27557   ,
         0.30899   ,  0.48497   ],
       [ 0.41251516,  0.18398405, -0.39532211, ..., -0.34461632,
         0.03530358, -0.20251337],
       [ 0.188321  , -0.008485  , -0.22926667, ..., -0.31138667,
         0.096795  ,  0.010395  ]])

In [130]:
print(Y)

[0 0 0 ... 1 1 1]


In [131]:
# Load data
import torch
import torch.nn as nn
X = torch.Tensor(X_train)
y = torch.LongTensor(Y)
print(X)
print(y)

tensor([[ 0.2108,  0.2028, -0.1510,  ..., -0.0332, -0.0079,  0.0236],
        [ 0.4142,  0.3068, -0.1923,  ..., -0.0792, -0.2794,  0.1208],
        [ 0.1732,  0.5176,  0.1253,  ..., -0.4029,  0.0755, -0.4325],
        ...,
        [ 0.6185,  0.6425, -0.4655,  ..., -0.2756,  0.3090,  0.4850],
        [ 0.4125,  0.1840, -0.3953,  ..., -0.3446,  0.0353, -0.2025],
        [ 0.1883, -0.0085, -0.2293,  ..., -0.3114,  0.0968,  0.0104]])
tensor([0, 0, 0,  ..., 1, 1, 1])


In [132]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
model = LogisticRegression()
model.fit(x_train,y_train)
ypred = model.predict(x_test)
print(accuracy_score(ypred, y_test))


0.7


In [133]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    
    #sigmoid transformation of the input 
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [134]:
n_samples, n_features = x_train.shape
lr = LogisticRegression(n_features)

In [135]:

num_epochs = 500
# Traning the model for large number of epochs to see better results  
learning_rate = 0.001
criterion = nn.BCELoss()                                
# We are working on lgistic regression so using Binary Cross Entropy
optimizer = torch.optim.SGD(lr.parameters(), lr=learning_rate)

In [136]:
for epoch in range(num_epochs):
    y_pred = lr(x_train)
    y_train = y_train.reshape(y_train.shape[0],1)

    loss = criterion(y_pred.float(), y_train.float())             
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch+1) % 20 == 0:                                         
        # printing loss values on every 10 epochs to keep track
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 20, loss = 0.6973
epoch: 40, loss = 0.6972
epoch: 60, loss = 0.6970
epoch: 80, loss = 0.6969
epoch: 100, loss = 0.6968
epoch: 120, loss = 0.6966
epoch: 140, loss = 0.6965
epoch: 160, loss = 0.6964
epoch: 180, loss = 0.6962
epoch: 200, loss = 0.6961
epoch: 220, loss = 0.6960
epoch: 240, loss = 0.6959
epoch: 260, loss = 0.6958
epoch: 280, loss = 0.6956
epoch: 300, loss = 0.6955
epoch: 320, loss = 0.6954
epoch: 340, loss = 0.6953
epoch: 360, loss = 0.6952
epoch: 380, loss = 0.6951
epoch: 400, loss = 0.6950
epoch: 420, loss = 0.6948
epoch: 440, loss = 0.6947
epoch: 460, loss = 0.6946
epoch: 480, loss = 0.6945
epoch: 500, loss = 0.6944


In [137]:
with torch.no_grad():
    y_predicted = lr(x_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 241.3000


In [153]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# Define logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.linear(x)
        out = self.sigmoid(out)
        return out

# Define hyperparameters
input_size = 50
output_size = 1
learning_rate = 0.01
num_epochs = 1000



In [158]:

# Define cross-validation
kf = KFold(n_splits=10, shuffle=True)
y=torch.LongTensor(Y)
# Train and evaluate model with cross-validation
acc_list = []
for train_index, test_index in kf.split(X):
    # Split data into train and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Define model and optimizer
    model = LogisticRegression(input_size, output_size)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    # Train the model
    for epoch in range(num_epochs):
        # Forward pass
        outputs = model(X_train)
        print(outputs.shape)
        print(y_train.shape)
        loss = nn.functional.cross_entropy(outputs, y_train)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Evaluate the model
    with torch.no_grad():
        outputs = model(X_test)
        _, predicted = torch.max(outputs.data, 1)
        accuracy = accuracy_score(y_test.numpy(), predicted.numpy())
        acc_list.append(accuracy)
print(acc_list)
# Compute the average accuracy over all folds
avg_acc = sum(acc_list) / len(acc_list)

print("Average accuracy: {:.2f}%".format(avg_acc*100))


torch.Size([2160, 1])
torch.Size([2160])


IndexError: Target 1 is out of bounds.